<a href="https://colab.research.google.com/github/kat-le/autogluon-kaggle/blob/main/CA_housing_price_autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use AutoGluon for Kaggle competitions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)



This tutorial will teach you how to use AutoGluon to become a serious Kaggle competitor without writing lots of code.
We first outline the general steps to use AutoGluon in Kaggle contests. Here, we assume the competition involves tabular data which are stored in one (or more) CSV files.

1) Run Bash command: pip install kaggle

2) Navigate to: https://www.kaggle.com/account and create an account (if necessary).
Then , click on "Create New API Token" and move downloaded file to this location on your machine: `~/.kaggle/kaggle.json`. For troubleshooting, see [Kaggle API instructions](https://www.kaggle.com/docs/api).

3) To download data programmatically: Execute this Bash command in your terminal:

`kaggle competitions download -c [COMPETITION]`

Here, [COMPETITION] should be replaced by the name of the competition you wish to enter.
Alternatively, you can download data manually: Just navigate to website of the Kaggle competition you wish to enter, click "Download All", and accept the competition's terms.

4) If the competition's training data is comprised of multiple CSV files, use [pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) to properly merge/join them into a single data table where rows = training examples, columns = features.

5) Run autogluon `fit()` on the resulting data table.

6) Load the test dataset from competition (again making the necessary merges/joins to ensure it is in the exact same format as the training data table), and then call autogluon `predict()`.  Subsequently use [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to load the competition's `sample_submission.csv` file into a DataFrame, put the AutoGluon predictions in the right column of this DataFrame, and finally save it as a CSV file via [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html). If the competition does not offer a sample submission file, you will need to create the submission file yourself by appropriately reformatting AutoGluon's test predictions.

7) Submit your predictions via Bash command:

`kaggle competitions submit -c [COMPETITION] -f [FILE] -m ["MESSAGE"]`

Here, [COMPETITION] again is the competition's name, [FILE] is the name of the CSV file you created with your predictions, and ["MESSAGE"] is a string message you want to record with this submitted entry. Alternatively, you can  manually upload your file of predictions on the competition website.

8) Finally, navigate to competition leaderboard website to see how well your submission performed!
It may take time for your submission to appear.



Below, we demonstrate how to do steps (4)-(6) in Python for a specific Kaggle competition: [ieee-fraud-detection](https://www.kaggle.com/c/ieee-fraud-detection/).
This means you'll need to run the above steps with `[COMPETITION]` replaced by `ieee-fraud-detection` in each command.  Here, we assume you've already completed steps (1)-(3) and the data CSV files are available on your computer. To begin step (4), we first load the competition's training data into Python:

In [1]:
!pip -q install -U pip
!pip -q install -U autogluon kaggle

import autogluon, sys, platform, os, pandas as pd, numpy as np
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
!nvidia-smi || true

DIR = "/content/california-house-prices/"
os.makedirs(DIR, exist_ok=True)
print("Working dir:", DIR)

Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Tue Oct 14 01:57:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   37C    P0             58W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |         

In [2]:
!pip uninstall -y torchaudio

In [3]:
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle (1).json
total 4
-rw------- 1 root root 65 Oct 14 01:58 kaggle.json


In [4]:
%cd /content
!kaggle competitions download -c california-house-prices -p "{DIR}"
!unzip -o -q "{DIR}/california-house-prices.zip" -d "{DIR}"
!ls -lh "{DIR}" | head -n 20

/content
  0% 0.00/29.5M [00:00<?, ?B/s]
100% 29.5M/29.5M [00:00<00:00, 1.51GB/s]
total 116M
-rw-r--r-- 1 root root  30M Mar 19  2021 california-house-prices.zip
-rw-r--r-- 1 root root 248K Mar 19  2021 sample_submission.csv
-rw-r--r-- 1 root root  35M Mar 19  2021 test.csv
-rw-r--r-- 1 root root  51M Mar 19  2021 train.csv


In [5]:
%cd /content
!curl -L -o example_kaggle_house.py \
  https://raw.githubusercontent.com/autogluon/autogluon/master/examples/automm/kaggle_california_house_price/example_kaggle_house.py
!sed -n '1,40p' example_kaggle_house.py

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6053  100  6053    0     0  18587      0 --:--:-- --:--:-- --:--:-- 18624
import pandas as pd
import numpy as np
import argparse
import os
import random
from autogluon.tabular import TabularPredictor
from autogluon.multimodal import MultiModalPredictor
import torch as th


def get_parser():
    parser = argparse.ArgumentParser(
        description='The Basic Example of AutoGluon for House Price Prediction.')
    parser.add_argument('--mode',
                        choices=['stack5',
                                 'weighted',
                                 'single',
                                 'single_bag5'],
                        default='weighted',
                        help='"stack5" means 5-fold stacking. "weighted" means weighted ensemble.'
                             ' "single" means use a single 

In [9]:
EXP="exp_weighted_ft"
!python3 example_kaggle_house.py --automm-mode mlp --mode single \
  --data_path "$DIR" --exp_path exp_single_mlp

2025-10-14 03:22:59.900238: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 03:22:59.918277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760412179.940386   43441 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760412179.947002   43441 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760412179.963756   43441 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [7]:
!ls -lh "$EXP"
!head -n 5 "$EXP/submission.csv"

total 4.6M
-rw-r--r-- 1 root root 4.6M Oct 14 01:07 learner.pkl
-rw-r--r-- 1 root root  21K Oct 14 01:04 metadata.json
drwxr-xr-x 6 root root 4.0K Oct 14 01:41 models
-rw-r--r-- 1 root root  620 Oct 14 01:04 predictor.pkl
drwxr-xr-x 4 root root 4.0K Oct 14 01:07 utils
-rw-r--r-- 1 root root    5 Oct 14 01:04 version.txt
head: cannot open 'exp_weighted_ft/submission.csv' for reading: No such file or directory
